## Instalujemy niezbędne biblioteki i pobieramy potrzebne pliki

In [8]:
!pip install spacy
#https://github.com/sdadas/polish-nlp-resources/releases/download/v1.0/word2vec.zip
!python -m spacy download pl_core_news_sm
!pip install gensim

^C
^C


## Wczytujemy dane

In [1]:
import os

# Zakładając, że pliki znajdują się w folderze 'data'
files = []
for i in range(1, 6):
    with open(f"data/kircholm{i}.txt", "r", encoding="utf-8") as file:
        files.append(file.read())

# Wydrukuj początek każdego tekstu, aby sprawdzić
for i, content in enumerate(files, start=1):
    print(f"Content of kircholm{i}: {content[:200]}...")  # Pierwsze 200 znaków

Content of kircholm1: https://www.gov.pl/web/lotwa/417-ta-rocznica-bitwy-pod-kircholmem


Z okazji 417-ej rocznicy bitwy pod Kircholmem kwiaty pod pomnikiem w Salaspils upamiętniającym zwycięstwo wojsk Hetmana Wielkiego Ja...
Content of kircholm2: https://ciekawostkihistoryczne.pl/2016/02/28/podobno-polacy-nie-potrafia-wykorzystywac-swoich-zwyciestw-udowodnimy-ci-ze-to-klamstwo/

27 września 1605 roku. Armia Rzeczpospolitej liczyła wówczas 3500...
Content of kircholm3: https://muzhp.pl/kalendarium/bitwa-pod-kircholmem
Stoczona pięć lat po rozpoczęciu polsko-szwedzkiego konfliktu o Inflanty bitwa pod Kircholmem (obecnie Salaspils na Łotwie) została uznana za jedno z ...
Content of kircholm4: https://pl.wikipedia.org/wiki/Bitwa_pod_Kircholmem

Bitwa pod Kircholmem (obecnie miejscowość Salaspils na Łotwie, 25 km na południowy wschód od Rygi) – bitwa stoczona 27 września 1605 w czasie polsko...
Content of kircholm5: https://wszystkoconajwazniejsze.pl/pepites/bitwa-pod-kircholmem-najwiekszy

## Preprocessing

In [6]:
import spacy

# Załaduj polski model spaCy
nlp = spacy.load("pl_core_news_sm")

# Funkcja preprocessująca tekst
def preprocess_text(text):
    doc = nlp(text)
    # Lematyzacja słów
    return [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]

# Przetwarzanie plików
processed_files = [preprocess_text(file_content) for file_content in files]

# Przykład pierwszego przetworzonego tekstu
for i, processed_text in enumerate(processed_files, start=1):
    print(f"Processed text for kircholm{i}: {processed_text[:10]}...")  # Pierwsze 10 słów

Processed text for kircholm1: ['https://www.gov.pl/web/lotwa/417-ta-rocznica-bitwy-pod-kircholm', '\n\n\n', 'okazja', '417', 'ej', 'rocznica', 'bitwa', 'Kircholm', 'kwiat', 'pomniek']...
Processed text for kircholm2: ['https://ciekawostkihistoryczne.pl/2016/02/28/podobno-polacy-nie-potrafia-wykorzystywac-swoich-zwyciestw-udowodnimy-ci-ze-to-klamstwo/', '\n\n', '27', 'wrzesień', '1605', 'Armia', 'Rzeczpospolita', 'liczyć', 'wówczas', '3500']...
Processed text for kircholm3: ['https://muzhp.pl/kalendarium/bitwa-pod-kircholm', '\n', 'stoczona', 'pięć', 'rok', 'rozpoczęć', 'polski', 'szwedzki', 'konflikt', 'Inflanty']...
Processed text for kircholm4: ['https://pl.wikipedia.org/wiki/Bitwa_pod_Kircholm', '\n\n', 'Bitwa', 'Kircholm', 'obecnie', 'miejscowość', 'Salaspils', 'Łotwa', '25', 'kilometr']...
Processed text for kircholm5: ['https://wszystkoconajwazniejsze.pl/pepites/bitwa-pod-kircholmem-najwiekszy-tryumf-husarii/', '\n\n', 'Bitwa', 'Kircholm', 'uznawać', 'duży', 'moment', 'chwała', '

## Przeprowadzenie Word Embedding z użyciem Word2Vec

In [9]:
from gensim.models import Word2Vec

# Trening modelu Word2Vec dla każdego pliku osobno
embedding_models = []
for i, processed_text in enumerate(processed_files, start=1):
    # Trening modelu Word2Vec dla przetworzonego tekstu
    model = Word2Vec([processed_text], vector_size=100, window=5, min_count=1, workers=4)
    embedding_models.append(model)

# Sprawdź przykładowe embeddingi dla pierwszego słowa
for i, model in enumerate(embedding_models, start=1):
    word = processed_files[i-1][0]  # Pierwsze słowo w każdym przetworzonym tekście
    vector = model.wv[word]
    print(f"Embedding for the word '{word}' in kircholm{i}: {vector[:5]}...")  # Pierwsze 5 wartości wektora

Embedding for the word 'https://www.gov.pl/web/lotwa/417-ta-rocznica-bitwy-pod-kircholm' in kircholm1: [ 0.00188934  0.00448157  0.0014389  -0.00537073  0.00981126]...
Embedding for the word 'https://ciekawostkihistoryczne.pl/2016/02/28/podobno-polacy-nie-potrafia-wykorzystywac-swoich-zwyciestw-udowodnimy-ci-ze-to-klamstwo/' in kircholm2: [ 0.00061226 -0.00014674 -0.00695949  0.00697505 -0.001054  ]...
Embedding for the word 'https://muzhp.pl/kalendarium/bitwa-pod-kircholm' in kircholm3: [-0.00695511 -0.00042382  0.00368951  0.00419012  0.00241677]...
Embedding for the word 'https://pl.wikipedia.org/wiki/Bitwa_pod_Kircholm' in kircholm4: [-0.0086343  -0.00527622  0.00360566 -0.00295995 -0.00749923]...
Embedding for the word 'https://wszystkoconajwazniejsze.pl/pepites/bitwa-pod-kircholmem-najwiekszy-tryumf-husarii/' in kircholm5: [-0.00611494  0.00471554 -0.00143119 -0.00521171 -0.00399123]...


## Wyniki zapisz do pliku json

In [11]:
import json
import os

# Upewnij się, że katalog /data/ istnieje
output_dir = "data"
os.makedirs(output_dir, exist_ok=True)

# Zapisz embeddingi do pliku JSON dla każdego tekstu
for i, model in enumerate(embedding_models, start=1):
    # Tworzenie słownika embeddingów
    embeddings_dict = {word: model.wv[word].tolist() for word in model.wv.index_to_key}

    # Ścieżka zapisu do pliku w katalogu /data/
    output_file = os.path.join(output_dir, f"kircholm{i}_embeddings.json")

    # Zapisz embeddings w formacie JSON
    with open(output_file, "w", encoding="utf-8") as json_file:
        json.dump(embeddings_dict, json_file, ensure_ascii=False, indent=4)

    print(f"Saved embeddings for kircholm{i} to file: {output_file}")

Saved embeddings for kircholm1 to file: data\kircholm1_embeddings.json
Saved embeddings for kircholm2 to file: data\kircholm2_embeddings.json
Saved embeddings for kircholm3 to file: data\kircholm3_embeddings.json
Saved embeddings for kircholm4 to file: data\kircholm4_embeddings.json
Saved embeddings for kircholm5 to file: data\kircholm5_embeddings.json
